In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_excel('weather.xls', usecols=range(1,6))

In [4]:
data

,outlook,temp.,hum.,windy,class
0,sunny,85,85,False,Don’t Play
1,sunny,80,90,True,Don’t Play
2,overcast,83,78,False,Play
3,rain,70,96,False,Play
4,rain,68,80,False,Play
5,rain,65,70,True,Don’t Play
6,overcast,64,65,True,Play
7,sunny,72,95,False,Don’t Play
8,sunny,69,70,False,Play
9,rain,75,80,False,Play


In [5]:
data.columns

Index(['outlook', 'temp.', 'hum.', 'windy', 'class'], dtype='object')

In [6]:
data.dtypes

outlook    object
temp.       int64
hum.        int64
windy        bool
class      object
dtype: object

In [168]:
data.sort_values(by=['temp.'])


,outlook,temp.,hum.,windy,class
6,overcast,64,65,True,Play
5,rain,65,70,True,Don’t Play
4,rain,68,80,False,Play
8,sunny,69,70,False,Play
3,rain,70,96,False,Play
13,rain,71,80,True,Don’t Play
7,sunny,72,95,False,Don’t Play
11,outlook,72,90,True,Play
9,rain,75,80,False,Play
10,sunny,75,70,True,Play


In [8]:
df['temp.']

6     64
5     65
4     68
8     69
3     70
13    71
7     72
11    72
9     75
10    75
1     80
12    81
2     83
0     85
Name: temp., dtype: int64

In [172]:
def find_most_freq_class(bin):
    #structure of bin: [(index, num_val, class)]
    set_class = []
    for val in bin:
        set_class.append(val[2])
    from collections import Counter
    unique_val = list(Counter(set_class).keys())
    count_val = list(Counter(set_class).values())
    ind_max = np.argmax(count_val)
    #if there`s several equal max 
    for idx, val in enumerate(count_val):
        #if there`s val that equals to our defined and have another index
        if val == count_val[ind_max] and idx != ind_max:
            print(count_val[ind_max])
            return False
    return unique_val[ind_max]
def discretize(numeric_clmn_idx, target_clmn_idx):
    #sort instances according to atributes values
    df = data.sort_values(by=[data.columns[numeric_clmn_idx]])
    #iter counter
    counter = 0
    bins = []
    temporary_bin = []
    for index, row in df.iterrows():
        num_val = row[numeric_clmn_idx]
        cls = row[target_clmn_idx]
        #first step when there`s nothing
        if counter == 0:
            # bin appereances as (index, num_val, class)
            temporary_bin.append( (index, num_val, cls))
        else: #that`s where real job starts
            if len(temporary_bin) == 1:
                # if numeric values are equal OR if classes are equal
                if temporary_bin[0][1] == num_val or temporary_bin[0][2] == cls:
                    temporary_bin.append( (index, num_val, cls))
                else:
                    bins.append(temporary_bin)
                    temporary_bin = []
                    temporary_bin.append( (index, num_val, cls))
            # if number of values in temporary_bin more than 1
            else:
                most_freq_class = find_most_freq_class(temporary_bin)
                #if there`s the most class
                if most_freq_class:
                    #if class or last num_val equals
                    if most_freq_class == cls or temporary_bin[-1][1] == num_val:
                        temporary_bin.append( (index, num_val, cls))
                    #if candidate value isn't appropriate 
                    else:
                        bins.append(temporary_bin)
                        temporary_bin = []
                        temporary_bin.append( (index, num_val, cls))
                #if there isn't the most class of temporary_bin, we'll just add new value to the bin
                else:
                    temporary_bin.append( (index, num_val, cls))
        counter += 1
    # FOR loop ended
    #in case if the last bin wasn't added
    if len(temporary_bin) >= 1:
        bins.append(temporary_bin)
    return bins

In [173]:
temperature_bins = discretize(1,4)
humidity_bins = discretize(2,4)

1


In [165]:
for val in temperature_bins:
    print(val)
    print('\n')

[(6, 64, 'Play')]


[(5, 65, 'Don’t Play')]


[(4, 68, 'Play'), (8, 69, 'Play'), (3, 70, 'Play')]


[(13, 71, 'Don’t Play'), (7, 72, 'Don’t Play'), (11, 72, 'Play')]


[(9, 75, 'Play'), (10, 75, 'Play')]


[(1, 80, 'Don’t Play')]


[(12, 81, 'Play'), (2, 83, 'Play')]


[(0, 85, 'Don’t Play')]




In [174]:
for val in humidity_bins:
    print(val)
    print('\n')

[(6, 65, 'Play')]


[(5, 70, 'Don’t Play'), (8, 70, 'Play'), (10, 70, 'Play'), (12, 75, 'Play'), (2, 78, 'Play'), (4, 80, 'Play'), (9, 80, 'Play'), (13, 80, 'Don’t Play')]


[(0, 85, 'Don’t Play'), (1, 90, 'Don’t Play'), (11, 90, 'Play'), (7, 95, 'Don’t Play')]


[(3, 96, 'Play')]




In [183]:
for idx_bin, each_bin in enumerate(temperature_bins):
    for val in each_bin:
        #val = (index, numeric_value, class)
        #data.loc(row_idx, column_name)
        data.loc[val[0], 'temp.'] = idx_bin

In [184]:
data

,outlook,temp.,hum.,windy,class
0,sunny,7,85,False,Don’t Play
1,sunny,5,90,True,Don’t Play
2,overcast,6,78,False,Play
3,rain,2,96,False,Play
4,rain,2,80,False,Play
5,rain,1,70,True,Don’t Play
6,overcast,0,65,True,Play
7,sunny,3,95,False,Don’t Play
8,sunny,2,70,False,Play
9,rain,4,80,False,Play


In [185]:
for idx_bin, each_bin in enumerate(humidity_bins):
    for val in each_bin:
        #val = (index, numeric_value, class)
        #data.loc(row_idx, column_name)
        data.loc[val[0], 'hum.'] = idx_bin

In [186]:
data

,outlook,temp.,hum.,windy,class
0,sunny,7,2,False,Don’t Play
1,sunny,5,2,True,Don’t Play
2,overcast,6,1,False,Play
3,rain,2,3,False,Play
4,rain,2,1,False,Play
5,rain,1,1,True,Don’t Play
6,overcast,0,0,True,Play
7,sunny,3,2,False,Don’t Play
8,sunny,2,1,False,Play
9,rain,4,1,False,Play


In [188]:
# SHOULD WE TOUCH CLASS WHEN DISCR?